In [1]:
# 导入必须的库
import numpy as np
import pandas as pd
from time import time
from IPython.display import display
# import pandas_profiling

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline

# 导入数据
data_train = pd.read_csv("train.csv")
data_store = pd.read_csv("store.csv")
data_test = pd.read_csv("test.csv")

# 显示第一条记录
display(data_train.head())

C:\Users\taili\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [2]:
#Null值
display(data_train.isnull().sum(),data_train.isnull().sum(),data_store.isnull().sum())

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

Store                          0
StoreType                      0
Assortment                     0
CompetitionDistance            3
CompetitionOpenSinceMonth    354
CompetitionOpenSinceYear     354
Promo2                         0
Promo2SinceWeek              544
Promo2SinceYear              544
PromoInterval                544
dtype: int64

In [3]:
#Null值处理
data_store.fillna(0,inplace=True)
data_test.fillna(1, inplace=True)

In [4]:
#Null值
display(data_train.isnull().sum(),data_train.isnull().sum(),data_store.isnull().sum())

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

Store            0
DayOfWeek        0
Date             0
Sales            0
Customers        0
Open             0
Promo            0
StateHoliday     0
SchoolHoliday    0
dtype: int64

Store                        0
StoreType                    0
Assortment                   0
CompetitionDistance          0
CompetitionOpenSinceMonth    0
CompetitionOpenSinceYear     0
Promo2                       0
Promo2SinceWeek              0
Promo2SinceYear              0
PromoInterval                0
dtype: int64

In [5]:
display(data_train.dtypes,data_test.dtypes)

Store             int64
DayOfWeek         int64
Date             object
Sales             int64
Customers         int64
Open              int64
Promo             int64
StateHoliday     object
SchoolHoliday     int64
dtype: object

Id                 int64
Store              int64
DayOfWeek          int64
Date              object
Open             float64
Promo              int64
StateHoliday      object
SchoolHoliday      int64
dtype: object

In [6]:
#日期格式转换
data_train['Date'] = pd.to_datetime(data_train['Date'])
data_test['Date'] = pd.to_datetime(data_test['Date'])

In [7]:
train = pd.merge(data_train, data_store, on='Store')
test = pd.merge(data_test, data_store, on='Store')

In [8]:
train = train.sort_values(['Date'],ascending = False)
ho_test = train[:6*7*1115]
ho_train = train[6*7*1115:]

In [9]:
# only use data of Sales>0 and Open is 1
ho_test = ho_test[(ho_test["Open"] != 0)&(ho_test["Sales"] > 0)]
ho_train = ho_train[(ho_train["Open"] != 0)&(ho_train["Sales"] > 0)]

In [10]:
def features_create(data):
    
    mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
    data.StoreType.replace(mappings, inplace=True)
    data.Assortment.replace(mappings, inplace=True)
    data.StateHoliday.replace(mappings, inplace=True)
    
    data['StateHoliday'] = data['StateHoliday'].astype('int')
    data['Assortment'] = data['Assortment'].astype('int')
    data['StoreType'] = data['StoreType'].astype('int')
    
    
    data['Year'] = data.Date.dt.year
    data['Month'] = data.Date.dt.month
    data['Day'] = data.Date.dt.day
    data['DayOfWeek'] = data.Date.dt.dayofweek
    data['WeekOfYear'] = data.Date.dt.weekofyear
    
    
    
    data['CompetitionOpen'] = 12 * (data.Year - data.CompetitionOpenSinceYear) +         (data.Month - data.CompetitionOpenSinceMonth)
    data['PromoOpen'] = 12 * (data.Year - data.Promo2SinceYear) +         (data.WeekOfYear - data.Promo2SinceWeek) / 4.0
    data['CompetitionOpen'] = data.CompetitionOpen.apply(lambda x: x if x > 0 else 0)        
    data['PromoOpen'] = data.PromoOpen.apply(lambda x: x if x > 0 else 0)
    
    
  
    month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun',              7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
    data['monthStr'] = data.Month.map(month2str)
    data.loc[data.PromoInterval == 0, 'PromoInterval'] = ''
    data['IsPromoMonth'] = 0
    for interval in data.PromoInterval.unique():
        if interval != '':
            for month in interval.split(','):
                data.loc[(data.monthStr == month) & (data.PromoInterval == interval), 'IsPromoMonth'] = 1

    return data

In [11]:
features_create(ho_train)
features_create(ho_test)
features_create(test)
print('Features creation finished')

Features creation finished


In [12]:
# drop the features of not help
ho_train.drop(['Date','Customers','Open','PromoInterval','monthStr'],axis=1,inplace =True)
ho_test.drop(['Date','Customers','Open','PromoInterval','monthStr'],axis=1,inplace =True)

In [13]:
ho_xtrain = ho_train.drop(['Sales'],axis=1 )
ho_ytrain = np.log1p(ho_train.Sales)
ho_xtest = ho_test.drop(['Sales'],axis=1 )
ho_ytest = np.log1p(ho_test.Sales)

In [14]:
xtest =test.drop(['Id','Date','Open','PromoInterval','monthStr'],axis = 1)

In [15]:
#定义评估函数
def rmspe(y_true, y_pred):
    loss = np.sqrt(np.mean(np.square(((y_true - y_pred) / y_true)), axis=0))
    return loss

def rmspe_xg(y_pred, y_true):
    return "rmspe", rmspe(np.expm1(y_true.get_label()),np.expm1(y_pred))

In [16]:
import xgboost as xgb
#XGboost参数设置
params = {"objective": "reg:squarederror",
          'learning_rate':0.1,
          "booster" : "gbtree",
          "eta": 0.03,
          "max_depth": 10,
          "min_child_weight":1,
          "subsample": 0.9,
          "colsample_bytree": 0.7,
          "gamma":0.8,
          "silent": 1,
          "seed": 10,
          "tree_method":"gpu_hist",
          "gpu_id":0
          }
num_boost_round = 10000

In [17]:
dtrain = xgb.DMatrix(ho_xtrain, ho_ytrain)
dvalid = xgb.DMatrix(ho_xtest, ho_ytest)
watchlist = [(dtrain, 'train'), (dvalid, 'eval')]

In [18]:
def grid_dict(param_test):
    combine_list = []
    key_nums = 0
    for i in param_test:
        key_nums += 1
        for j in param_test[i]:
            combine_list.append({i:j})

    from itertools import combinations
    result = []
    for i in combinations(combine_list, 2):
        tmp_dict = {}
        for j in list(i):
            for key in j:
                tmp_dict[key] = j[key]
        if len(tmp_dict) == key_nums:
            result.append(tmp_dict)
    
    return result

def train_model(params):
    print("Train a XGBoost model")
    start = time()
    gbm = xgb.train(params, dtrain, num_boost_round, evals=watchlist,early_stopping_rounds=100, feval=rmspe_xg, verbose_eval=False)
    end = time()
    print('Training time is {:2f} s.'.format(end-start))
    
    print("validating")
    ho_xtest.sort_index(inplace=True) 
    ho_ytest.sort_index(inplace=True) 
    yhat = gbm.predict(xgb.DMatrix(ho_xtest))
    error = rmspe(np.expm1(ho_ytest), np.expm1(yhat))

    print('RMSPE: {:.6f}'.format(error))
    return error

def grid_search_xg(params, params_grid):
    iter_num = 0
    grid_list = grid_dict(params_grid)
    print('Iter nums is {:1f}.'.format(len(grid_list)))
    min_error = 100
    result_params = []
    for i in grid_list:
        for key in i:
            params[key] = i[key]
        print(params)
        error = train_model(params)
        if error<min_error:
            result_params = i
            min_error = error
        iter_num += 1
        print('Iter No is {:1f}.'.format(iter_num))
    
    return result_params        

In [19]:
param_grid = {'max_depth':range(7,15,1), 'min_child_weight':range(1,10,1)}

In [20]:
grid_search_xg(params, param_grid)

Iter nums is 72.000000.
{'objective': 'reg:squarederror', 'learning_rate': 0.1, 'booster': 'gbtree', 'eta': 0.03, 'max_depth': 7, 'min_child_weight': 1, 'subsample': 0.9, 'colsample_bytree': 0.7, 'gamma': 0.8, 'silent': 1, 'seed': 10, 'tree_method': 'gpu_hist', 'gpu_id': 0}
Train a XGBoost model
Training time is 108.307038 s.
validating
RMSPE: 0.139434
Iter No is 1.000000.
{'objective': 'reg:squarederror', 'learning_rate': 0.1, 'booster': 'gbtree', 'eta': 0.03, 'max_depth': 7, 'min_child_weight': 2, 'subsample': 0.9, 'colsample_bytree': 0.7, 'gamma': 0.8, 'silent': 1, 'seed': 10, 'tree_method': 'gpu_hist', 'gpu_id': 0}
Train a XGBoost model


KeyboardInterrupt: 

In [125]:
print("Make predictions on the test set")
dtest = xgb.DMatrix(xtest)
test_probs = gbm.predict(dtest)

Make predictions on the test set


In [126]:
# model1  kaggle private score 0.12647
result = pd.DataFrame({"Id": test['Id'], 'Sales': np.expm1(test_probs)})
result.to_csv("Rossmann_submission_1.csv", index=False)

In [ ]:
# model2 kaggle private score 0.11756
result = pd.DataFrame({"Id": test['Id'], 'Sales': np.expm1(test_probs*0.995)})
result.to_csv("Rossmann_submission_2.csv", index=False)

In [ ]:
# model3 kaggle private score 0.11292
result = pd.DataFrame({"Id": test['Id'], 'Sales': np.expm1(test_probs*W_test)})
result.to_csv("Rossmann_submission_3.csv", index=False)